In [ ]:
%%capture
!pip install -r requirements.txt --quiet

In [ ]:
%%capture
# Importing necessary libraries and handling warnings
import warnings
warnings.filterwarnings('ignore')  # Ignore warnings to keep notebook clean

import torch
import torch.nn as nn
from transformers import (
    pipeline,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoConfig,
)
from langchain.llms import HuggingFacePipeline

from IPython.display import Markdown
import re
import random
import pandas as pd

## Ingest
This section is dedicated to loading and preparing data for processing. Here, we specifically handle markdown files, loading them from a local directory.

In [ ]:
# Loading markdown files from the specified directory
# Using a glob pattern to select all markdown files recursively
loader_md = DirectoryLoader("./content/", glob="**/*.md")
try:
    md_data = loader_md.load()
    print("Markdown files loaded successfully:", len(md_data))
except Exception as e:
    print("Failed to load markdown files:", e)

## Document Splitters
In this section, we define how documents are split into manageable parts for further analysis or processing. This is crucial for handling large texts efficiently.

In [ ]:
%%capture
# Importing necessary libraries and handling warnings
import warnings
warnings.filterwarnings('ignore')  # Ignore warnings to keep notebook clean

import torch
import torch.nn as nn
from transformers import (
    pipeline,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoConfig,
)
from langchain.llms import HuggingFacePipeline

from IPython.display import Markdown
import re
import random
import pandas as pd

## Vector Stores

In [ ]:
%%capture
# Importing necessary libraries and handling warnings
import warnings
warnings.filterwarnings('ignore')  # Ignore warnings to keep notebook clean

import torch
import torch.nn as nn
from transformers import (
    pipeline,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoConfig,
)
from langchain.llms import HuggingFacePipeline

from IPython.display import Markdown
import re
import random
import pandas as pd

In [ ]:
%%capture
# Importing necessary libraries and handling warnings
import warnings
warnings.filterwarnings('ignore')  # Ignore warnings to keep notebook clean

import torch
import torch.nn as nn
from transformers import (
    pipeline,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoConfig,
)
from langchain.llms import HuggingFacePipeline

from IPython.display import Markdown
import re
import random
import pandas as pd

In [ ]:
# Assuming 'rdocs' is a list of Document objects as shown above
def print_retrieved_documents(documents):
    for idx, doc in enumerate(documents, start=1):
        print(f"Document {idx}:")
        print(f"Source: {doc.metadata.get('source')}\n")
        # Splitting the content into paragraphs for better readability
        paragraphs = doc.page_content.split("\n\n")
        for paragraph in paragraphs:
            print(paragraph)
        print("\n" + "-" * 80 + "\n")  # Add a separator line between documents

# Retrieve documents using a retriever from vectordb
retriever = vectordb.as_retriever(search_kwargs={"k": 2})
rdocs = retriever.get_relevant_documents("chatbot")

# Improved print of retrieved documents
print_retrieved_documents(rdocs)

In [ ]:
# Query to retrieve similar chunks
query = "Kafka?"

# Retrieve similar chunks based on relevance. We only retrieve 'k' most similar chunks
similar_chunks = vectordb.similarity_search_with_relevance_scores(query, k=3)

# Format document to text format
retrieved_text = [chunk[0].page_content for chunk in similar_chunks]
relevance_score = [chunk[1] for chunk in similar_chunks]

# Store and print as a dataframe
retrieved_chunks = pd.DataFrame(
    list(zip(retrieved_text, relevance_score)),
    columns=["Retrieved Chunks", "Relevance Score"],
)
with pd.option_context("display.max_colwidth", None):
    display(retrieved_chunks)

## LLM Chain

In [ ]:
%%capture
# Importing necessary libraries and handling warnings
import warnings
warnings.filterwarnings('ignore')  # Ignore warnings to keep notebook clean

import torch
import torch.nn as nn
from transformers import (
    pipeline,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoConfig,
)
from langchain.llms import HuggingFacePipeline

from IPython.display import Markdown
import re
import random
import pandas as pd

In [ ]:
chain.invoke({"question": "Why the application needs Kafka?"})